# **Ad copy**

Few shot prompting.



### 1. Setup requirements and dependencies




In [ ]:
%pip install transformers==4.38.2 --quiet
%pip install datasets==2.18.0  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

### 2. Load the sample hugging face dataset

In [ ]:
huggingface_ad_copy_dataset = "jaykin01/advertisement-copy"
dataset = load_dataset(huggingface_ad_copy_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
for i in range(0, 3):
  print(dataset['train'][i]["product"])
  print(dataset['train'][i]["description"])
  print(dataset['train'][i]["ad"])

  print("********************************")

 Harem pants
 A style of pants with a dropped crotch, loose-fitting legs, and a gathered waistband for a unique, bohemian look.
Discover Harem Pants! Unique, stylish bohemian vibes with a dropped crotch & loose legs. Comfy meets chic - elevate your wardrobe. Limited stock - shop now!
********************************
 Fringe skirt
 A skirt featuring fringe detailing on the bottom, creating movement and fun.
Introducing our fabulous Fringe Skirt! Step out in style with eye-catching fringe detailing that adds flair and movement. Perfect for any occasion, create unforgettable memories with this chic piece.
********************************
 Gingham dress
 A dress featuring a two-toned checkered pattern, often associated with picnics and summery outfits.
Introducing the Gingham Dress: Timeless & Chic! 💕 Step into summer with this must-have, two-toned checkered dress. From picnics to parties, it's your go-to look. Shop now for unbeatable style!
********************************


In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoModelForSeq2SeqLM, T5ForConditionalGeneration
import torch

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = '[PAD]'

test_indices = [100, 110, 459, 560, 990, 1000 ]
sample_indices = [501, 525, 560, 760, 880, 990]
dash_line = '-'.join('' for _ in range(100))
prompt = ''
for i, index in enumerate(test_indices):
    if index < len(dataset['train']):
        description = dataset['train'][index]["description"]
        ad = dataset['train'][index]["ad"]
        for sample_index in sample_indices:

                prompt += f"""
                Description: {dataset['train'][sample_indices[i]]["description"]}
                ad: {dataset['train'][sample_indices]["ad"]}



                """
        prompt += f"""
                Description: {description}
                ad: {ad}

                """

        input_ids = tokenizer(prompt, return_tensors="pt").input_ids
        output = model.generate(
            input_ids,
            max_length=150
        )[0]

        generated_text = tokenizer.decode(output, skip_special_tokens=True)

        print(dash_line)
        print(f'Example {i + 1}')
        print(dash_line)
        print(f'INPUT PROMPT:\n{description}')
        print(dash_line)
        print(f'BASELINE AD:\n{ad}')
        print(dash_line)
        print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{generated_text}\n')



tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1935 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
 A wide, shallow neckline that extends from shoulder to shoulder, creating a classic and feminine look.
---------------------------------------------------------------------------------------------------
BASELINE AD:
Shop now! Discover elegance redefined with our timeless Boatneck Blouse - the perfect blend of a classic wide neckline & feminine charm, crafted for your ultimate style statement!
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
['Boatneck Blouse']

---------------------------------------------------------------------------------------------------
Example 2
---------------------------------------------------------------------------------------------------
I